<a href="https://colab.research.google.com/github/rustproofpanda0/test_task/blob/main/preprocessing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [2]:
pd.set_option('display.max_columns', None)  # or 1000


#отображение всехстрок таблицы или небольшой их части
# переклчал по необходимости

pd.set_option('display.max_rows', 1000)  # or 1000
# pd.set_option('display.max_rows', None)

In [3]:
energy_df = pd.read_csv('energy.csv', sep='\t')
plan_df_full = pd.read_csv('plan.csv', sep='\t')

In [4]:
energy_df # при просмотре файлов замечаем, что в energy.csv необходимо заменить запятые на точки

,Дата,Характерный день,Торговый час,Характерный час,"График ППП, МВтч",Объем Торгового графика (скорректированное плановое почасовое потребление) МВт*час,Объем покупки МВт*час,"В т.ч. потери, отнесенные к объему покупки","Фактическое собственное потребление, МВтч","Покупка (+) / Продажа (-) на БР (BR+/-), МВтч","Объем покупки на БР (BR+), МВтч","Объем продажи на БР (BR-), МВтч"
0,01.01.2015,выходной,0,минимум,"0,45","0,45","0,463","0,013","0,362","-0,088",0,"0,088"
1,01.01.2015,выходной,1,минимум,"0,45","0,45","0,463","0,013","0,355","-0,095",0,"0,095"
2,01.01.2015,выходной,2,минимум,"0,45","0,45","0,463","0,013","0,355","-0,095",0,"0,095"
3,01.01.2015,выходной,3,минимум,"0,45","0,45","0,463","0,013","0,355","-0,095",0,"0,095"
4,01.01.2015,выходной,4,минимум,"0,45","0,45","0,463","0,013","0,364","-0,086",0,"0,086"
...,...,...,...,...,...,...,...,...,...,...,...,...
26299,31.12.2017,выходной,19,полупик,0,0,0,0,"0,159",0,0,0
26300,31.12.2017,выходной,20,полупик,0,0,0,0,"0,152",0,0,0
26301,31.12.2017,выходной,21,полупик,0,0,0,0,"0,154",0,0,0
26302,31.12.2017,выходной,22,минимум,0,0,0,0,"0,146",0,0,0


In [5]:
energy_df.dtypes # просматриваем типы данных в столбцах

Дата                                                                                  object
Характерный день                                                                      object
Торговый час                                                                           int64
Характерный час                                                                       object
График ППП, МВтч                                                                      object
Объем Торгового графика (скорректированное плановое почасовое потребление) МВт*час    object
Объем покупки МВт*час                                                                 object
В т.ч. потери, отнесенные к объему покупки                                            object
Фактическое собственное потребление, МВтч                                             object
Покупка (+) / Продажа (-) на БР (BR+/-), МВтч                                         object
Объем покупки на БР (BR+), МВтч                                       

In [6]:
str_mask = energy_df.dtypes == 'object' # для столбцов с типом object (все их них str) меняем запятые на точки
for col in energy_df.columns[str_mask]:
    energy_df[col] = energy_df[col].str.replace(',', '.')

In [7]:
cols_to_float = energy_df.columns[4:]
for col in cols_to_float:
    print(col)
    energy_df[col] = energy_df[col].astype(float)

График ППП, МВтч
Объем Торгового графика (скорректированное плановое почасовое потребление) МВт*час
Объем покупки МВт*час
В т.ч. потери, отнесенные к объему покупки
Фактическое собственное потребление, МВтч
Покупка (+) / Продажа (-) на БР (BR+/-), МВтч
Объем покупки на БР (BR+), МВтч
Объем продажи на БР (BR-), МВтч


ValueError: ignored

In [8]:
# пока что переведём в float то, что можно перевести
# код выше падает на последнем столбце, следовательно проблемы только в нём

cols_to_float = energy_df.columns[4:-1]
for col in cols_to_float:
    energy_df[col] = energy_df[col].astype(float)

In [9]:
# Просмотрев теблицу, увидим, что проблема последнего столбца
# в символах ' -     ' , которые стоят, предположительно, вместо нулей.
# проверим это

dash_tst_df = energy_df[energy_df['Объем продажи на БР (BR-), МВтч'] == ' -      ']

switch = True
for i in dash_tst_df['Покупка (+) / Продажа (-) на БР (BR+/-), МВтч'] >= 0:
    if (not i):
        print('less then 0')
        switch = False

# условие не сработало, следовательно на месте ' -     ' должны быть нули

if (switch):
    energy_df.replace(' -      ', '0.0', inplace=True)
    energy_df['Покупка (+) / Продажа (-) на БР (BR+/-), МВтч'] = energy_df[
              'Покупка (+) / Продажа (-) на БР (BR+/-), МВтч'].astype(float)

In [10]:
print(energy_df['Характерный час'].unique()) # проверяем остальные столбцы на наличие неправильных данных
print(energy_df['Характерный день'].unique())
print(energy_df['Торговый час'].unique())
# пока что оставим категориальные данные в таком виде

['минимум' 'полупик' 'пик']
['выходной' 'рабочий день']
[ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23]


In [11]:
energy_df.dtypes # проверяем типы данных полученной таблицы

Дата                                                                                   object
Характерный день                                                                       object
Торговый час                                                                            int64
Характерный час                                                                        object
График ППП, МВтч                                                                      float64
Объем Торгового графика (скорректированное плановое почасовое потребление) МВт*час    float64
Объем покупки МВт*час                                                                 float64
В т.ч. потери, отнесенные к объему покупки                                            float64
Фактическое собственное потребление, МВтч                                             float64
Покупка (+) / Продажа (-) на БР (BR+/-), МВтч                                         float64
Объем покупки на БР (BR+), МВтч                             

In [12]:
energy_df['Дата'] = pd.to_datetime(energy_df['Дата'], dayfirst=True)

In [13]:
energy_df

,Дата,Характерный день,Торговый час,Характерный час,"График ППП, МВтч",Объем Торгового графика (скорректированное плановое почасовое потребление) МВт*час,Объем покупки МВт*час,"В т.ч. потери, отнесенные к объему покупки","Фактическое собственное потребление, МВтч","Покупка (+) / Продажа (-) на БР (BR+/-), МВтч","Объем покупки на БР (BR+), МВтч","Объем продажи на БР (BR-), МВтч"
0,2015-01-01,выходной,0,минимум,0.45,0.45,0.463,0.013,0.362,-0.088,0.0,0.088
1,2015-01-01,выходной,1,минимум,0.45,0.45,0.463,0.013,0.355,-0.095,0.0,0.095
2,2015-01-01,выходной,2,минимум,0.45,0.45,0.463,0.013,0.355,-0.095,0.0,0.095
3,2015-01-01,выходной,3,минимум,0.45,0.45,0.463,0.013,0.355,-0.095,0.0,0.095
4,2015-01-01,выходной,4,минимум,0.45,0.45,0.463,0.013,0.364,-0.086,0.0,0.086
...,...,...,...,...,...,...,...,...,...,...,...,...
26299,2017-12-31,выходной,19,полупик,0.00,0.00,0.000,0.000,0.159,0.000,0.0,0
26300,2017-12-31,выходной,20,полупик,0.00,0.00,0.000,0.000,0.152,0.000,0.0,0
26301,2017-12-31,выходной,21,полупик,0.00,0.00,0.000,0.000,0.154,0.000,0.0,0
26302,2017-12-31,выходной,22,минимум,0.00,0.00,0.000,0.000,0.146,0.000,0.0,0


In [14]:
plan_df_full

,DateTime,product_1_plan,product_1_plan_cumsum,product_1_fact,product_1_fact_cumsum,product_2_status,product_2_plan,product_2_plan_cumsum,product_2_fact,product_2_fact_cumsum,product_2_accomplished,product_3_status,product_3_plan,product_3_plan_cumsum,product_3_fact,product_3_fact_cumsum,product_3_accomplished,product_4_status,product_4_plan,product_4_plan_cumsum,product_4_fact,product_4_fact_cumsum,product_4_accomplished
0,2015-09-01,*1500,1500,1317.735,1317.735,NaN,100.0,100.0,530.009,530.009,530.009000,NaN,25.0,25.0,81.011,81.011,324.044000,NaN,5.0,5.0,7.997,7.997,159.940000
1,2015-09-02,800,2300,755.753,2073.488,NaN,950.0,1050.0,979.918,1509.927,143.802571,NaN,25.0,50.0,28.002,109.013,218.026000,NaN,5.0,10.0,5.600,13.597,135.970000
2,2015-09-03,800,3100,729.503,2802.991,NaN,950.0,2000.0,1022.824,2532.751,126.637550,NaN,25.0,75.0,28.018,137.031,182.708000,ТО,NaN,NaN,6.004,19.601,196.010000
3,2015-09-04,800,3900,836.252,3639.243,NaN,950.0,2950.0,969.786,3502.537,118.730068,NaN,25.0,100.0,33.785,170.816,170.816000,NaN,5.0,15.0,8.008,27.609,184.060000
4,2015-09-05,800,4700,707.373,4346.616,NaN,950.0,3900.0,948.310,4450.847,114.124282,NaN,25.0,125.0,38.857,209.673,167.738400,NaN,5.0,20.0,5.596,33.205,166.025000
5,2015-09-06,800,5500,917.865,5264.481,NaN,950.0,4850.0,1028.242,5479.089,112.970907,NaN,25.0,150.0,9.986,219.659,146.439333,NaN,5.0,25.0,5.632,38.837,155.348000
6,2015-09-07,800,6300,655.914,5920.395,NaN,950.0,5800.0,666.870,6145.959,105.964810,NaN,25.0,175.0,24.027,243.686,139.249143,NaN,5.0,30.0,4.827,43.664,145.546667
7,2015-09-08,800,7100,624.445,6544.840,ТО,NaN,NaN,218.012,6363.971,109.723638,NaN,25.0,200.0,29.159,272.845,136.422500,NaN,5.0,35.0,2.010,45.674,130.497143
8,2015-09-09,800,7900,772.839,7317.679,NaN,900.0,6700.0,621.966,6985.937,104.267716,NaN,25.0,225.0,35.852,308.697,137.198667,NaN,5.0,40.0,7.234,52.908,132.270000
9,2015-09-10,800,8700,783.485,8101.164,NaN,950.0,7650.0,952.594,7938.531,103.771647,ТО,NaN,NaN,33.478,342.175,152.077778,NaN,5.0,45.0,11.640,64.548,143.440000


In [15]:
# достаём план производства из plan.csv
plan_columns = ['DateTime'] 
for col in plan_df_full.columns:
    if (col.find('plan') != -1): # проверяем названия колонок на наличие подстроки "plan"
        plan_columns.append(col)

plan_df = plan_df_full[plan_columns].copy() #план производства

In [16]:
plan_df

,DateTime,product_1_plan,product_1_plan_cumsum,product_2_plan,product_2_plan_cumsum,product_3_plan,product_3_plan_cumsum,product_4_plan,product_4_plan_cumsum
0,2015-09-01,*1500,1500,100.0,100.0,25.0,25.0,5.0,5.0
1,2015-09-02,800,2300,950.0,1050.0,25.0,50.0,5.0,10.0
2,2015-09-03,800,3100,950.0,2000.0,25.0,75.0,NaN,NaN
3,2015-09-04,800,3900,950.0,2950.0,25.0,100.0,5.0,15.0
4,2015-09-05,800,4700,950.0,3900.0,25.0,125.0,5.0,20.0
5,2015-09-06,800,5500,950.0,4850.0,25.0,150.0,5.0,25.0
6,2015-09-07,800,6300,950.0,5800.0,25.0,175.0,5.0,30.0
7,2015-09-08,800,7100,NaN,NaN,25.0,200.0,5.0,35.0
8,2015-09-09,800,7900,900.0,6700.0,25.0,225.0,5.0,40.0
9,2015-09-10,800,8700,950.0,7650.0,NaN,NaN,5.0,45.0


In [17]:
plan_df.dtypes

DateTime                  object
product_1_plan            object
product_1_plan_cumsum     object
product_2_plan           float64
product_2_plan_cumsum    float64
product_3_plan           float64
product_3_plan_cumsum    float64
product_4_plan           float64
product_4_plan_cumsum    float64
dtype: object

In [18]:
plan_df[['product_1_plan', 'product_1_plan_cumsum']].astype(float)

ValueError: ignored

In [19]:
plan_df

,DateTime,product_1_plan,product_1_plan_cumsum,product_2_plan,product_2_plan_cumsum,product_3_plan,product_3_plan_cumsum,product_4_plan,product_4_plan_cumsum
0,2015-09-01,*1500,1500,100.0,100.0,25.0,25.0,5.0,5.0
1,2015-09-02,800,2300,950.0,1050.0,25.0,50.0,5.0,10.0
2,2015-09-03,800,3100,950.0,2000.0,25.0,75.0,NaN,NaN
3,2015-09-04,800,3900,950.0,2950.0,25.0,100.0,5.0,15.0
4,2015-09-05,800,4700,950.0,3900.0,25.0,125.0,5.0,20.0
5,2015-09-06,800,5500,950.0,4850.0,25.0,150.0,5.0,25.0
6,2015-09-07,800,6300,950.0,5800.0,25.0,175.0,5.0,30.0
7,2015-09-08,800,7100,NaN,NaN,25.0,200.0,5.0,35.0
8,2015-09-09,800,7900,900.0,6700.0,25.0,225.0,5.0,40.0
9,2015-09-10,800,8700,950.0,7650.0,NaN,NaN,5.0,45.0


In [20]:
# for i, j in zip(plan_df['product_1_plan'], plan_df['product_1_plan_cumsum']):
#     print(f'{i} with type: {type(i)}     {j}with type {type(j)}')

# уберём звёздочки
for i in range(plan_df.shape[0]):
    tmp1 = plan_df.loc[i, 'product_1_plan']
    if (type(tmp1) == str):
        plan_df.loc[i, 'product_1_plan'] = tmp1.replace('*', '')

    tmp2 = plan_df.loc[i, 'product_1_plan_cumsum']
    if (type(tmp2) == str):
        plan_df.loc[i, 'product_1_plan_cumsum'] = tmp2.replace('*', '')



In [21]:
plan_df[['product_1_plan', 'product_1_plan_cumsum']].astype(float)

ValueError: ignored

In [22]:
plan_df[plan_df['product_1_plan_cumsum'] == '']

,DateTime,product_1_plan,product_1_plan_cumsum,product_2_plan,product_2_plan_cumsum,product_3_plan,product_3_plan_cumsum,product_4_plan,product_4_plan_cumsum
331,2016-07-28,NaN,,950.0,14775.0,NaN,NaN,3.0,67.0


In [23]:
plan_df.replace(to_replace='', value=np.nan, inplace=True)

In [24]:
plan_df['product_1_plan'] = plan_df['product_1_plan'].astype(float)
plan_df['product_1_plan_cumsum'] = plan_df['product_1_plan_cumsum'].astype(float)

In [25]:
plan_df.dtypes

DateTime                  object
product_1_plan           float64
product_1_plan_cumsum    float64
product_2_plan           float64
product_2_plan_cumsum    float64
product_3_plan           float64
product_3_plan_cumsum    float64
product_4_plan           float64
product_4_plan_cumsum    float64
dtype: object

In [26]:
"""
При замене nan в таблице возможны три случая:
1) product_i_plan = nan, product_i_plan_cumsum = nan
    - делаем product_i_plan = 0.0, product_i_plan_cumsum = product_(i - 1)_plan_cumsum
2) product_i_plan = nan, product_i_plan_cumsum = число
    - делаем product_i_plan = product_i_plan_cumsum - product_(i - 1)_plan_cumsum
3) product_i_plan = число, product_i_plan_cumsum = nan
    - делаем product_i_plan_cumsum = product_(i - 1)_plan_cumsum + product_i_plan

"""

na_mask = plan_df.isna()
for i in range(plan_df.shape[0]):
    na_mask_row = na_mask.loc[i, :]
    for n_prod in range(1, 5):
        plan = 'product_%i_plan' % n_prod
        cumsum = 'product_%i_plan_cumsum' % n_prod

        if (na_mask_row[plan] and na_mask_row[cumsum]):
            plan_df.loc[i, plan] = 0.0
            plan_df.loc[i, cumsum] = plan_df.loc[i - 1, cumsum]

        elif (na_mask_row[plan] and not na_mask_row[cumsum]):
            plan_df.loc[i, plan] = plan_df.loc[i, cumsum] - plan_df.loc[i - 1, cumsum]

        elif (not na_mask_row[plan] and na_mask_row[cumsum]):
            plan_df.loc[i, cumsum] = plan_df.loc[i - 1, cumsum] + plan_df.loc[i, plan]

In [27]:
plan_df.isna().sum(axis=0) # проверяем остались ли nan в таблице

DateTime                 0
product_1_plan           0
product_1_plan_cumsum    0
product_2_plan           0
product_2_plan_cumsum    0
product_3_plan           0
product_3_plan_cumsum    0
product_4_plan           0
product_4_plan_cumsum    0
dtype: int64

In [28]:
plan_df['DateTime'] = pd.to_datetime(plan_df['DateTime'])

In [29]:
energy_df

,Дата,Характерный день,Торговый час,Характерный час,"График ППП, МВтч",Объем Торгового графика (скорректированное плановое почасовое потребление) МВт*час,Объем покупки МВт*час,"В т.ч. потери, отнесенные к объему покупки","Фактическое собственное потребление, МВтч","Покупка (+) / Продажа (-) на БР (BR+/-), МВтч","Объем покупки на БР (BR+), МВтч","Объем продажи на БР (BR-), МВтч"
0,2015-01-01,выходной,0,минимум,0.45,0.45,0.463,0.013,0.362,-0.088,0.0,0.088
1,2015-01-01,выходной,1,минимум,0.45,0.45,0.463,0.013,0.355,-0.095,0.0,0.095
2,2015-01-01,выходной,2,минимум,0.45,0.45,0.463,0.013,0.355,-0.095,0.0,0.095
3,2015-01-01,выходной,3,минимум,0.45,0.45,0.463,0.013,0.355,-0.095,0.0,0.095
4,2015-01-01,выходной,4,минимум,0.45,0.45,0.463,0.013,0.364,-0.086,0.0,0.086
...,...,...,...,...,...,...,...,...,...,...,...,...
26299,2017-12-31,выходной,19,полупик,0.00,0.00,0.000,0.000,0.159,0.000,0.0,0
26300,2017-12-31,выходной,20,полупик,0.00,0.00,0.000,0.000,0.152,0.000,0.0,0
26301,2017-12-31,выходной,21,полупик,0.00,0.00,0.000,0.000,0.154,0.000,0.0,0
26302,2017-12-31,выходной,22,минимум,0.00,0.00,0.000,0.000,0.146,0.000,0.0,0


In [30]:
plan_df

,DateTime,product_1_plan,product_1_plan_cumsum,product_2_plan,product_2_plan_cumsum,product_3_plan,product_3_plan_cumsum,product_4_plan,product_4_plan_cumsum
0,2015-09-01,1500.0,1500.0,100.0,100.0,25.0,25.0,5.0,5.0
1,2015-09-02,800.0,2300.0,950.0,1050.0,25.0,50.0,5.0,10.0
2,2015-09-03,800.0,3100.0,950.0,2000.0,25.0,75.0,0.0,10.0
3,2015-09-04,800.0,3900.0,950.0,2950.0,25.0,100.0,5.0,15.0
4,2015-09-05,800.0,4700.0,950.0,3900.0,25.0,125.0,5.0,20.0
5,2015-09-06,800.0,5500.0,950.0,4850.0,25.0,150.0,5.0,25.0
6,2015-09-07,800.0,6300.0,950.0,5800.0,25.0,175.0,5.0,30.0
7,2015-09-08,800.0,7100.0,0.0,5800.0,25.0,200.0,5.0,35.0
8,2015-09-09,800.0,7900.0,900.0,6700.0,25.0,225.0,5.0,40.0
9,2015-09-10,800.0,8700.0,950.0,7650.0,0.0,225.0,5.0,45.0


In [32]:
energy_df.to_csv('energy_preprocessed.csv', sep='\t')
plan_df.to_csv('plan_preprocessed.csv', sep='\t')